## Capture Tweets from Twitter and save it to PostgreSQL database.

Objective is to capture the tweets from Twitter for a specific keyword to analyze the trend. we can convert this to a simple python script and run it in the background.

This information can be used to find different kinds of patterns like number of tweets, retweets, followers and text mining on actual tweet.

## Prerequisites

#### install postgresql 
( reference - https://ramanakothi.com/1262/)
Create database twitter from command line
1. createdb twitter 
2. initdb -D twitter
3. pg_ctl -D twitter -l logfile start

#### Create user twitter from command line execute  - psql twitter
1. create user twitter with password 'twitter'; 
2. alter role twitter CREATEROLE CREATEDB;)

#### setup twitter account and get API Key

To start with, you will need to have a Twitter developer account and obtain credentials (i.e. API key, API secret, Access token and Access token secret) on the to access the Twitter API, following these steps:

Create a Twitter developer account if you do not already have one from : https://developer.twitter.com/
Go to https://developer.twitter.com/en/apps and log in with your Twitter user account.
Click “Create an app”
Fill out the form, and click “Create”
A pop up window will appear for reviewing Developer Terms. Click the “Create” button again.
In the next page, click on “Keys and Access Tokens” tab, and copy your “API key” and “API secret” from the Consumer API keys section.
Scroll down to Access token & access token secret section and click “Create”. Then copy your “Access token” and “Access token secret”

### Install required python modules

In [ ]:
!pip install tweepy, psycopg2, sqlalchemy

In [2]:
import psycopg2
import tweepy 
import json

#### Create keys.py file, needs to enter values in below block for Twitter API and execute it.

In [ ]:
%%writefile keys.py
## create a file keys.py in the working directory and add below attrributes
#### Variables that contains the user credentials from Twitter API (setup twitter account and get API Key).
access_token = "ENTER ACCESS TOKEN"
access_token_secret = "ENTER ACCESS TOKEN SECRET"
consumer_key = "ENTER CONSUMER KEY"
consumer_secret = "ENTER CONSUMER SECRET"

#### Postgres Credentials

dbname='twitter'
user ='twitter'
password='twitter'
host='localhost'
port=5432

In [11]:
## importing data from the keys files which contains twitter and db parameters.
from keys import *

 #### This function gets the consumer key, consumer secret key, access token and access token secret given by the app created in your Twitter account and authenticate them with Tweepy.

In [4]:
def autorize_twitter_api():
    auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_token, access_token_secret)
    return auth

#### This function open a connection with an already created database and creates a new table to store tweets related to a subject specified by the user


In [12]:
def create_tweets_table(term_to_search):
    #Connect to Twitter Database created in Postgres
    conn_twitter = psycopg2.connect(dbname=dbname, user=user, password=password, host=host, port=port)

    #Create a cursor to perform database operations
    cursor_twitter = conn_twitter.cursor()

    #with the cursor now, create two tables, users twitter and the corresponding table according to the selected topic
    cursor_twitter.execute("CREATE TABLE IF NOT EXISTS twitter_users (user_id VARCHAR PRIMARY KEY, user_name VARCHAR);")
    
    query_create = "CREATE TABLE IF NOT EXISTS %s (id SERIAL, created_at timestamp, tweet text NOT NULL, user_id VARCHAR, \
                    retweetcount int, PRIMARY KEY(id), FOREIGN KEY(user_id) REFERENCES twitter_users(user_id));" %("tweets_"+term_to_search)
    
    cursor_twitter.execute(query_create)
    
    #Commit changes
    conn_twitter.commit()
    
    #Close cursor and the connection
    cursor_twitter.close()
    conn_twitter.close()
    return

#### This function open a connection with an already created database and inserts into corresponding table tweets related to the selected topic

In [6]:
def store_tweets_in_table(term_to_search, user_id, created_at, tweet, user_name, retweetcount):
    #Connect to Twitter Database created in Postgres
    conn_twitter = psycopg2.connect(dbname=dbname, user=user, password=password, host=host, port=port)
    
    #Create a cursor to perform database operations
    cursor_twitter = conn_twitter.cursor()

    #with the cursor now, insert tweet into table
    cursor_twitter.execute("INSERT INTO twitter_users (user_id, user_name) VALUES (%s, %s) ON CONFLICT(user_id) DO NOTHING;", (user_id, user_name))
    
    cursor_twitter.execute("INSERT INTO %s (created_at, tweet, user_id, retweetcount) VALUES (%%s, %%s, %%s, %%s);" %('tweets_'+term_to_search), 
                           (created_at, tweet, user_id, retweetcount))
    
    #Commit changes
    conn_twitter.commit()
    
    #Close cursor and the connection
    cursor_twitter.close()
    conn_twitter.close()
    return

#### This is the class that streams data and writes to table. It can be updated to handle exceptions.

In [7]:
class MyStreamListener(tweepy.StreamListener):
    def on_data(self, raw_data):

        try:
            global term_to_search
            
            data = json.loads(raw_data)            
            
            #Obtain all the variables to store in each column
            user_id = data['user']['id_str']
            created_at = data['created_at']
            tweet = data['text']
            user_name = data['user']['screen_name']
            retweetcount = data['retweet_count']
            
            #Store them in the corresponding table in the database
            store_tweets_in_table(term_to_search, user_id, created_at, tweet, user_name, retweetcount)
            
        except Exception as e:
            #e = sys.exc_info()[1]
            #print(e.args[0])
            pass
    
    def on_error(self, status_code):
        if status_code == 420:
            #returning False in on_error disconnects the stream
            return False

#### You can use any search term, I have used Coronavirus to know tweet patterns.

In [ ]:
if __name__ == "__main__": 
    #Creates the table for storing the tweets
    term_to_search = "coronavirus"
    create_tweets_table(term_to_search)
    
    #Connect to the streaming twitter API
    api = tweepy.API(wait_on_rate_limit_notify=True)
    
    #Stream the tweets
    streamer = tweepy.Stream(auth=autorize_twitter_api(), listener=MyStreamListener(api=api))
    streamer.filter(languages=["en"], track=[term_to_search])

#### you can copy this code into a py script and run it in the background.

In [14]:
%%writefile Twit_Postgres.py
#!/usr/bin/env python

import psycopg2
import tweepy 
import json
from keys import *



def autorize_twitter_api():
    auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_token, access_token_secret)
    return auth



def create_tweets_table(term_to_search):
    #Connect to Twitter Database created in Postgres
    conn_twitter = psycopg2.connect(dbname=dbname, user=user, password=password, host=host, port=port)

    #Create a cursor to perform database operations
    cursor_twitter = conn_twitter.cursor()

    #with the cursor now, create two tables, users twitter and the corresponding table according to the selected topic
    cursor_twitter.execute("CREATE TABLE IF NOT EXISTS twitter_users (user_id VARCHAR PRIMARY KEY, user_name VARCHAR);")
    
    query_create = "CREATE TABLE IF NOT EXISTS %s (id SERIAL, created_at timestamp, tweet text NOT NULL, user_id VARCHAR,                     retweetcount int, PRIMARY KEY(id), FOREIGN KEY(user_id) REFERENCES twitter_users(user_id));" %("tweets_"+term_to_search)
    
    cursor_twitter.execute(query_create)
    
    #Commit changes
    conn_twitter.commit()
    
    #Close cursor and the connection
    cursor_twitter.close()
    conn_twitter.close()
    return

def store_tweets_in_table(term_to_search, user_id, created_at, tweet, user_name, retweetcount):
    #Connect to Twitter Database created in Postgres
    conn_twitter = psycopg2.connect(dbname=dbname, user=user, password=password, host=host, port=port)
    
    #Create a cursor to perform database operations
    cursor_twitter = conn_twitter.cursor()

    #with the cursor now, insert tweet into table
    cursor_twitter.execute("INSERT INTO twitter_users (user_id, user_name) VALUES (%s, %s) ON CONFLICT(user_id) DO NOTHING;", (user_id, user_name))
    
    cursor_twitter.execute("INSERT INTO %s (created_at, tweet, user_id, retweetcount) VALUES (%%s, %%s, %%s, %%s);" %('tweets_'+term_to_search), 
                           (created_at, tweet, user_id, retweetcount))
    
    #Commit changes
    conn_twitter.commit()
    
    #Close cursor and the connection
    cursor_twitter.close()
    conn_twitter.close()
    return

class MyStreamListener(tweepy.StreamListener):
    def on_data(self, raw_data):

        try:
            global term_to_search
            
            data = json.loads(raw_data)            
            
            #Obtain all the variables to store in each column
            user_id = data['user']['id_str']
            created_at = data['created_at']
            tweet = data['text']
            user_name = data['user']['screen_name']
            retweetcount = data['retweet_count']
            
            #Store them in the corresponding table in the database
            store_tweets_in_table(term_to_search, user_id, created_at, tweet, user_name, retweetcount)
            
        except Exception as e:
            #e = sys.exc_info()[1]
            #print(e.args[0])
            pass
    
    def on_error(self, status_code):
        if status_code == 420:
            #returning False in on_error disconnects the stream
            return False


if __name__ == "__main__": 
    #Creates the table for storing the tweets
    term_to_search = "coronavirus"
    create_tweets_table(term_to_search)
    
    #Connect to the streaming twitter API
    api = tweepy.API(wait_on_rate_limit_notify=True)
    
    #Stream the tweets
    streamer = tweepy.Stream(auth=autorize_twitter_api(), listener=MyStreamListener(api=api))
    streamer.filter(languages=["en"], track=[term_to_search])

Writing Twit_Postgres.py
